In [1]:
import tensorflow as tf
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from tensorflow.keras.models import model_from_json
import copy

In [3]:
from keras.models import load_model

model_json_file = "model.json"
model_weights_file = "emotion_detect.h5"
with open(model_json_file, "r") as json_file:
    loaded_model_json = json_file.read()
    classifier = model_from_json(loaded_model_json)
    classifier.load_weights(model_weights_file)

In [3]:
from keras.models import load_model

model = load_model('FR_prune_LRF.keras')

In [7]:
#!pip install tensorflow-model-optimization

In [4]:
import tensorflow_model_optimization as tfmot
tfmot.__version__

'0.8.0'

## Pruning

In [6]:
model_for_pruning = tfmot.sparsity.keras.prune_low_magnitude(model)

ValueError: Please initialize `Prune` with a supported layer. Layers should either be supported by the PruneRegistry (built-in keras layers) or should be a `PrunableLayer` instance, or should has a customer defined `get_prunable_weights` method. You passed: <class 'keras.layers.core.tf_op_layer.TFOpLambda'>

## Laod dataset:

In [7]:
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
IMG_SIZE = 224

In [8]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split


def produce_dataset(num_classes):
    images_set = []
    labels = []
    for i in range(num_classes):
        data_path = os.path.join("data", f"{str(i)}")
        for images in os.listdir(data_path):
            img = cv2.imread(os.path.join(data_path, images))
            faces = face_cascade.detectMultiScale(img, 1.3, 5)
            for (x, y, w, h) in faces:
                roi_color = img[y:y+h, x:x+w]
                resized_array = cv2.resize(roi_color, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_NEAREST)
                images_set.append(resized_array)
                labels.append(i+1)

    return images_set, labels

num_of_classes = 4
X, y = produce_dataset(num_of_classes) 
label_bin = LabelBinarizer()
y = label_bin.fit_transform(y)
X = np.array(X)
y = np.array(y)


test_size = 0.10
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

X_test = X_test / 255.0

In [9]:
# building data generator 

from keras.preprocessing.image import ImageDataGenerator

batch_size = 128
base_path = "images/images/"


train_datagen = ImageDataGenerator(rescale = 1.0/255.0,
                                  width_shift_range = 0.1,
                                   height_shift_range = 0.1,
                                   rotation_range = 20,
                                   horizontal_flip = True)

validation_datagen = ImageDataGenerator(rescale= 1.0/255)

train_generator = train_datagen.flow_from_directory(os.path.join(base_path, "train"),
                                                    target_size=(56,56),
                                                    color_mode="grayscale",
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=True)

validation_generator = validation_datagen.flow_from_directory(os.path.join(base_path, "validation"),
                                                    target_size=(56,56),
                                                    color_mode="grayscale",
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=False)

Found 28821 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.


## Quantization

In [6]:
classifier.save("./saved_modelfer/")

INFO:tensorflow:Assets written to: ./saved_modelfer/assets


INFO:tensorflow:Assets written to: ./saved_modelfer/assets


In [5]:
model.save("./saved_prunes_LRF/")

INFO:tensorflow:Assets written to: ./saved_prunes_LRF/assets


INFO:tensorflow:Assets written to: ./saved_prunes_LRF/assets


In [10]:
def representative_dataset():
    for data in tf.data.Dataset.from_tensor_slices(X_test).batch(1).take(100):
        yield [tf.cast(data, tf.float32)]

In [12]:
converter = tf.lite.TFLiteConverter.from_saved_model("./saved_prunes_LRF")
tflite_model = converter.convert()

In [13]:
converter = tf.lite.TFLiteConverter.from_saved_model("./saved_prunes_LRF")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
tflite_quant_model = converter.convert()

In [14]:
len(tflite_model)

4036248

In [15]:
len(tflite_quant_model)

1294336

In [20]:
with open("tflite_model_fer.tflite", "wb") as f:
    f.write(tflite_model)

In [21]:
with open("tflite_quant_model_fer.tflite", "wb") as f:
    f.write(tflite_quant_model)

In [16]:
def apply_quantization_to_dense(layer):
  if isinstance(layer, tf.keras.layers.Dense):
    return tfmot.quantization.keras.quantize_annotate_layer(layer)
  return layer

In [17]:
annotated_model = tf.keras.models.clone_model(
    model,
    clone_function=apply_quantization_to_dense,
)

In [18]:
quant_aware_model = tfmot.quantization.keras.quantize_apply(annotated_model)

## Training Quantization aware model

In [19]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
            rescale=1./255,
            width_shift_range=0.1,
            height_shift_range=0.1,
            horizontal_flip=True, 
            rotation_range=15,
            fill_mode = 'nearest')


datagen_val = ImageDataGenerator(rescale=1./255)

batch_size = 16

train_generator = datagen.flow(X_train, 
                               y_train, 
                               batch_size=batch_size,
                               shuffle=True
                               )

validation_generator = datagen_val.flow(X_val,
                                        y_val,
                                       batch_size = batch_size,
                                       shuffle = True)

print ("train images generator",len(train_generator))
print("Validation images generator:", len(validation_generator))

train images generator 34
Validation images generator: 8


In [20]:
optimizer1 = tf.keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=True,
    name='Adam',
)

quant_aware_model.compile(optimizer=optimizer1,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

#quant_aware_model.summary()

In [21]:
import time
start=time.time()

quant_aware_model.fit(train_generator,
                    batch_size = batch_size,
                    epochs=1,
                    shuffle = True,
                    validation_data= validation_generator)

print("total time", time.time()-start,"second")

34/34 [==============================] - 15s 243ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.0021 - val_accuracy: 1.0000
total time 15.470043897628784 second


In [22]:
quant_aware_model.evaluate(X_test, y_test)

4/4 [==============================] - 1s 208ms/step - loss: 0.0029 - accuracy: 1.0000


[0.0028632930479943752, 1.0]

In [23]:
X_train = X_train / 255.
def representative_data_gen():
    for input_data in tf.data.Dataset.from_tensor_slices(X_train.astype("float32")).batch(1).take(100):
        yield [input_data]

In [24]:
converter = tf.lite.TFLiteConverter.from_keras_model(quant_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8  # or tf.float16
converter.inference_output_type = tf.uint8  # or tf.float16
converter.representative_dataset = representative_data_gen
tflite_qaware_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\dlr22\AppData\Local\Temp\tmpbo4lagul\assets


INFO:tensorflow:Assets written to: C:\Users\dlr22\AppData\Local\Temp\tmpbo4lagul\assets
C:\Users\dlr22\anaconda3\envs\py310\lib\site-packages\tensorflow\lite\python\convert.py:789: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


In [25]:
len(tflite_qaware_model)

1294584

In [35]:
with open("tflite_qaware_model.tflite", 'wb') as f:
    f.write(tflite_qaware_model)

In [36]:
import tensorflow as tf

model_path = "tflite_qaware_model.tflite"
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Get tensor details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
tensor_details = interpreter.get_tensor_details()

# Calculate memory usage
total_memory = 0
for tensor in tensor_details:
    tensor_shape = tensor['shape']
    tensor_dtype = tensor['dtype']
    # Calculate size in bytes
    tensor_size = tf.dtypes.as_dtype(tensor_dtype).size * tf.math.reduce_prod(tensor_shape).numpy()
    total_memory += tensor_size

print(f"Total memory required for inference: {total_memory / 1024:.2f} KB")

RuntimeError: tensorflow/lite/kernels/kernel_util.cc:320 scale_diff / output_scale <= 0.02 was not true.Node number 110 (FULLY_CONNECTED) failed to prepare.Failed to apply the default TensorFlow Lite delegate indexed at 0.